In [6]:
import requests
import os
import pandas as pd
import io
import time
import json
import copy
import boto3.session
key = os.environ.get("USDAKEY")
bucket = os.environ.get("CROP_BUCKET")
profile = os.environ.get("AWS_LP")

In [7]:
s3 = (
    boto3.session.Session(profile_name=profile)
    .resource('s3')
)

In [10]:
states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO',
    'CT', 'DE', 'FL', 'GA', 'HI', 'ID',
    'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 
    'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 
    'MO', 'MT', 'NV', 'NE', 'NH', 'NJ', 
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
    'OR', 'PA', 'RI', 'SC', 'SD', 'TN',
    'TX', 'UT', 'VT', 'VA', 'WA', 'WV',
    'WI', 'WY'
]

In [11]:
def state_info(state):
    
    state_info = []
    
    for st in states:
        s = (
            requests.get(
            f'''http://quickstats.nass.usda.gov/api/api_GET/?key={key}&
            group_desc=INCOME&commodity_desc=COMMODITY+TOTALS&
            statisticcat_desc=SALES&unit_desc=$&state_alpha={st}&format=json''')
            .json()
        )
        state_info.append(s)
        time.sleep(3)
    
    return(state_info)

In [12]:
si = state_info(states)

In [13]:
def get_nested_data(init_list):
    
    select_data = []
    
    for state_info in init_list:
        i = state_info['data']
        select_data.append(i)
    
    return(select_data)

def change_column(nested_list):
    
    sd = copy.deepcopy(nested_list)
    
    final_ls = []
    
    for item in sd:
        for i in item:
            i['cv_per'] = i.pop("CV (%)")
        final_ls.append(item)
    
    return(final_ls)

In [14]:
gnd = get_nested_data(si)

In [15]:
cc = change_column(get_nested_data(si))

In [16]:
def write_files(ls):
        
    for i, j in enumerate(ls):
        with open(f'../data/state_{i}.jsonl', 'w') as filehandle:
            [filehandle.write('%s\n' % item) for item in j]

In [17]:
write_files(cc)

In [18]:
%%bash
for i in ../data/*.jsonl; do
     sed -i "s/'/\"/g" $i
done

In [19]:
def get_files():
    
    path = '../data'
    fl = []
    files = os.listdir(path)
    [fl.append(f) for f in files if 'state' in f]
    
    return(fl)


In [20]:
gf = get_files()

In [21]:
def write_to_s3(ls):
    
    for file in ls:
        s3.Object(bucket, f'crop-data/{file}').upload_file(f'../data/{file}')

In [22]:
write_to_s3(gf)